In [2]:
#Ishank Jain
#libraries
from __future__ import print_function
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD   #not used
from scipy.ndimage import rotate
from keras.datasets import mnist
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from PIL import Image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
%matplotlib notebook

In [4]:
def resize(images):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((32, 32))
        #img = np.reshape(img, (32, 32, 1))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

In [5]:
def rotates(images, deg):
    rotated = []
    for img in images:
        img = Image.fromarray(img)
        # Use PIL or other library of the sort to rotate
        img = rotate(img, deg, reshape=False)
        #img = np.array(img)
        img = np.reshape(img, (32, 32, 1))
        rotated.append(img)
    return np.array(rotated)

In [ ]:
batch_size = 150
num_classes = 10
epochs = 1

# input image dimensions to reshape data
#img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#if K.image_data_format() == 'channels_first':
#    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#    input_shape = (1, img_rows, img_cols)
#else:
#    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#    input_shape = (img_rows, img_cols, 1)

#converting data to 32x32 and normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = resize(x_train)
x_test = resize(x_test)
x_train = rotates(x_train, 0)
#normalize data
x_train /= 255
x_test /= 255

#list of degree to rotate image
degrees=list(range(-45,45,15))
for deg in degrees: 
    
    x_test = rotates(x_test, deg)

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    #block 1
    model.add(Conv2D(64,(3, 3),
                 activation='relu', 
                 padding='same',
                 ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 2
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   # 
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   #
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #FC layer and flattening data from 3D to 2D 
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    #Softmax layer
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

    history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    #print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    plt.plot(deg,score[1],'bo')
